In [1]:
import sys
import numpy as np
import chess.variant
import chess.pgn
import random
import pandas
import berserk
import webbrowser
import time
import os
import re
from collections import defaultdict

In [2]:
# https://stackoverflow.com/questions/2267362/how-to-convert-an-integer-in-any-base-to-a-string

import string

digs = string.digits + string.ascii_letters


def int2base(x, base, length):
    if x < 0:
        sign = -1
    elif x == 0:
        return digs[0]
    else:
        sign = 1

    x *= sign
    digits = []

    while x:
        digits.append(digs[int(x % base)])
        x = int(x / base)

    if sign < 0:
        digits.append('-')

    while len(digits) < length: digits.extend(["0"])

    return list(map(lambda x: int(x), digits))


FILE_MAP = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
RANK_MAP = {'1': 7, '2': 6, '3': 5, '4': 4, '5': 3, '6': 2, '7': 1, '8': 0}

RESIDUALS_MAP = {0.875 : 7, 0.75 : 6, 0.625 : 5, 0.5 : 4,0.375 : 3, 0.25 : 2, 0.125 : 1, 0.0 : 0}
PIECE_MAP = {"r" : -4, "n" : -2, "b": -3, "q" : -5, "k" : -6, "p" : -1, "R" : 4, "N" : 2, "B": 3, "Q" : 5, "K" : 6, "P" : 1}

FILE_MAP_REVERSE = {v: k for k, v in FILE_MAP.items()}
RANK_MAP_REVERSE = {v: k for k, v in RANK_MAP.items()}

PIECE_MAP = {"p" : 1, "P" : -1, "R" : -4, "r" : 4, "N" : -2, "n" : 2, "B" : -3, "b" : 3, "Q" : -5, "q" : 5, "K" : -6, "k" : 6}


PROMOTIONS = {'a2a1q': 0,
             'a2a1r': 11,
             'a2a1k': 12,
             'a2a1b': 13,
             'a2a1n': 14,
             'a2b1q': 15,
             'a2b1r': 19,
             'a2b1k': 20,
             'a2b1b': 21,
             'a2b1n': 22,
             'a7a8q': 23,
             'a7a8r': 25,
             'a7a8k': 26,
             'a7a8b': 28,
             'a7a8n': 29,
             'a7b8q': 30,
             'a7b8r': 31,
             'a7b8k': 33,
             'a7b8b': 34,
             'a7b8n': 35,
             'b2a1q': 37,
             'b2a1r': 38,
             'b2a1k': 39,
             'b2a1b': 41,
             'b2a1n': 42,
             'b2b1q': 43,
             'b2b1r': 44,
             'b2b1k': 46,
             'b2b1b': 47,
             'b2b1n': 49,
             'b2c1q': 50,
             'b2c1r': 51,
             'b2c1k': 52,
             'b2c1b': 53,
             'b2c1n': 55,
             'b7a8q': 57,
             'b7a8r': 58,
             'b7a8k': 59,
             'b7a8b': 60,
             'b7a8n': 61,
             'b7b8q': 62,
             'b7b8r': 65,
             'b7b8k': 76,
             'b7b8b': 77,
             'b7b8n': 78,
             'b7c8q': 79,
             'b7c8r': 84,
             'b7c8k': 85,
             'b7c8b': 86,
             'b7c8n': 87,
             'c2b1q': 88,
             'c2b1r': 90,
             'c2b1k': 91,
             'c2b1b': 93,
             'c2b1n': 94,
             'c2c1q': 95,
             'c2c1r': 96,
             'c2c1k': 98,
             'c2c1b': 99,
             'c2c1n': 100,
             'c2d1q': 102,
             'c2d1r': 103,
             'c2d1k': 104,
             'c2d1b': 106,
             'c2d1n': 107,
             'c7b8q': 108,
             'c7b8r': 109,
             'c7b8k': 111,
             'c7b8b': 112,
             'c7b8n': 114,
             'c7c8q': 115,
             'c7c8r': 116,
             'c7c8k': 117,
             'c7c8b': 118,
             'c7c8n': 120,
             'c7d8q': 122,
             'c7d8r': 123,
             'c7d8k': 124,
             'c7d8b': 125,
             'c7d8n': 126,
             'd2c1q': 127,
             'd2c1r': 130,
             'd2c1k': 141,
             'd2c1b': 142,
             'd2c1n': 143,
             'd2d1q': 149,
             'd2d1r': 150,
             'd2d1k': 151,
             'd2d1b': 152,
             'd2d1n': 153,
             'd2e1q': 155,
             'd2e1r': 156,
             'd2e1k': 158,
             'd2e1b': 159,
             'd2e1n': 160,
             'd7c8q': 161,
             'd7c8r': 163,
             'd7c8k': 164,
             'd7c8b': 165,
             'd7c8n': 167,
             'd7d8q': 168,
             'd7d8r': 169,
             'd7d8k': 171,
             'd7d8b': 172,
             'd7d8n': 173,
             'd7e8q': 174,
             'd7e8r': 176,
             'd7e8k': 177,
             'd7e8b': 179,
             'd7e8n': 180,
             'e2d1q': 181,
             'e2d1r': 182,
             'e2d1k': 183,
             'e2d1b': 184,
             'e2d1n': 185,
             'e2e1q': 187,
             'e2e1r': 188,
             'e2e1k': 189,
             'e2e1b': 190,
             'e2e1n': 191,
             'e2f1q': 195,
             'e2f1r': 200,
             'e2f1k': 206,
             'e2f1b': 207,
             'e2f1n': 208,
             'e7d8q': 214,
             'e7d8r': 215,
             'e7d8k': 217,
             'e7d8b': 218,
             'e7d8n': 220,
             'e7e8q': 221,
             'e7e8r': 223,
             'e7e8k': 224,
             'e7e8b': 225,
             'e7e8n': 226,
             'e7f8q': 228,
             'e7f8r': 229,
             'e7f8k': 230,
             'e7f8b': 232,
             'e7f8n': 233,
             'f2e1q': 234,
             'f2e1r': 236,
             'f2e1k': 237,
             'f2e1b': 238,
             'f2e1n': 239,
             'f2f1q': 240,
             'f2f1r': 241,
             'f2f1k': 242,
             'f2f1b': 244,
             'f2f1n': 245,
             'f2g1q': 246,
             'f2g1r': 247,
             'f2g1k': 248,
             'f2g1b': 249,
             'f2g1n': 250,
             'f7e8q': 252,
             'f7e8r': 253,
             'f7e8k': 254,
             'f7e8b': 255,
             'f7e8n': 260,
             'f7f8q': 264,
             'f7f8r': 265,
             'f7f8k': 271,
             'f7f8b': 272,
             'f7f8n': 273,
             'f7g8q': 279,
             'f7g8r': 280,
             'f7g8k': 282,
             'f7g8b': 283,
             'f7g8n': 285,
             'g2f1q': 286,
             'g2f1r': 289,
             'g2f1k': 290,
             'g2f1b': 291,
             'g2f1n': 293,
             'g2g1q': 294,
             'g2g1r': 295,
             'g2g1k': 296,
             'g2g1b': 297,
             'g2g1n': 298,
             'g2h1q': 299,
             'g2h1r': 301,
             'g2h1k': 302,
             'g2h1b': 303,
             'g2h1n': 304,
             'g7f8q': 305,
             'g7f8r': 306,
             'g7f8k': 307,
             'g7f8b': 309,
             'g7f8n': 310,
             'g7g8q': 311,
             'g7g8r': 312,
             'g7g8k': 313,
             'g7g8b': 314,
             'g7g8n': 315,
             'g7h8q': 317,
             'g7h8r': 318,
             'g7h8k': 319,
             'g7h8b': 325,
             'g7h8n': 328,
             'h2g1q': 329,
             'h2g1r': 330,
             'h2g1k': 336,
             'h2g1b': 337,
             'h2g1n': 338,
             'h2h1q': 344,
             'h2h1r': 345,
             'h2h1k': 347,
             'h2h1b': 348,
             'h2h1n': 350,
             'h7g8q': 351,
             'h7g8r': 352,
             'h7g8k': 354,
             'h7g8b': 355,
             'h7g8n': 356,
             'h7h8q': 358,
             'h7h8r': 359,
             'h7h8k': 361,
             'h7h8b': 362,
             'h7h8n': 363}

REVERSE_PROMOTIONS = {v: k for k, v in PROMOTIONS.items()}

def get_move_from_action(action):    
    if action in REVERSE_PROMOTIONS:
        move = REVERSE_PROMOTIONS[action]

    else:
        move = int2base(action, 8, 4)
        move = FILE_MAP_REVERSE[move[0]]+""+RANK_MAP_REVERSE[move[1]]+""+FILE_MAP_REVERSE[move[2]]+""+RANK_MAP_REVERSE[move[3]]
        
    return move


def get_action_from_move(move):

    action = -1
    if len(move) > 4:
        action = PROMOTIONS[move]
    else:
        x1, y1, x2, y2 = FILE_MAP[move[0]],RANK_MAP[move[1]],FILE_MAP[move[2]],RANK_MAP[move[3]]
        action = x1 + y1 * 8 + x2 * 8 ** 2 + y2 * 8 ** 3
    
    return action

In [3]:
#list pgn game files
pgn_games = os.listdir("C:\\Users\\jerne\\Downloads\\AntiChessGames")

In [4]:
all_games_array = []
for pgn_game in pgn_games:
    month_games = open("C:\\Users\\jerne\\Downloads\\AntiChessGames\\"+pgn_game)
    txt = month_games.read()
    array = txt.split("\n\n")
    all_games_array.append(array)
    month_games.close()

In [5]:
def from_canonical_to_actual(canonical_board):
    c_b = canonical_board
    pos = c_b[5:]
    ply = c_b[:5]

    board = []
    final_board = np.zeros((8,8)).astype(int)
    counter = 0
    i = 0
    for _ in range(len(pos)-5):
        if i >= len(pos):
            break
        piece = 0
        if pos[i] == "-":
            piece = pos[i]+pos[i+1]
            i = i + 1
        else:
            piece = pos[i]
        i = i + 1
        board.append(int(piece))

    for j in range(0,64,8):
        final_board[counter] = board[j:j+8]
        counter += 1

    square_at_index = 0
    dict_board = {}
    for i in reversed(range(0, 8)):
        for j in range(0, 8):
            piece = final_board[i][j]

            if piece != 0:

                dict_board[square_at_index] = chess.Piece.from_symbol(REVERSE_PIECE_MAP[piece])

            square_at_index = square_at_index + 1
           
    
    return dict_board

In [100]:
def count_moves(moves_dict,color,previous_moves,player_turn):
    moves_list = []
    final_moves = []
    final_final_moves = []
    for key,value in moves_dict.items():

        moves_list.append((key,value[0]/sum(value),sum(value)))

    moves_list.sort(key=lambda x: x[1],reverse=True)
    
    move_t = -1
    prob_w_t = -1
    prob_b_t = -1
    if player_turn == 0:
        move_t = 100000
        prob_w_t = 0.50
        prob_b_t = 0.55
    elif player_turn == 1:
        move_t = 20000
        prob_w_t = 0.50
        prob_b_t = 0.55
    elif player_turn == 2:
        move_t = 4000
        prob_w_t = 0.489
        prob_b_t = 0.55
    else:
        move_t = 800
        prob_w_t = 0.489
        prob_b_t = 0.55
    

    for item in moves_list:
        if color == "white":
            if (item[1] > prob_w_t) and (item[2] > move_t):
                final_moves.append((item[0],item[1],item[2]))
        else:
            if (item[1] < prob_b_t) and (item[2] > move_t):
                final_moves.append((item[0],item[1],item[2]))
                
    for move in final_moves:
        move1 = move[0]
        move2 = ""
        if player_turn == 0:
            move2 = move1
        elif player_turn == 1:
            move2 = move1.split(" ")[0]
        elif player_turn == 2:
            move2 = move1.split(" ")[0:2]
            move2 = " ".join(move2)

        else:
            move2 = move1.split(" ")[0:3]
            move2 = " ".join(move2)
        if move2 in previous_moves:
            
            final_final_moves.append(move[0])
        
    return moves_list,final_moves,final_final_moves

In [101]:
mlw1,fmlw1,ffmlw1 = count_moves(first_move_white,"white",['Nh3','g4','c4','e3','Na3','g3','b3'],0)
mlb1,fmlb1,ffmlb1 = count_moves(first_move_black,"black",ffmlw1,1)
mlw2,fmlw2,ffmlw2 = count_moves(second_move_white,"white",ffmlb1,2)
mlb2,fmlb2,ffmlb2 = count_moves(second_move_black,"black",ffmlw2,3)

In [102]:
def opening_book_first_white(book):
    board = chess.variant.AntichessBoard()
    total_moves = sum([move[2] for move in book])
    probs = np.zeros(4096)
    move_count = 0
    pre_moves = []
    for move in book:
        prob = move[2]/total_moves
        pre_moves.append((get_action_from_move(str(board.parse_san(move[0]))),prob))
    
    for pre_move in pre_moves:
        probs[pre_move[0]] = pre_move[1]
    return probs,board

In [103]:
fmlw1

[('Nh3', 0.5712763079410298, 613598),
 ('g4', 0.5593962228771996, 511977),
 ('c4', 0.5575896103276872, 671128),
 ('e3', 0.5400545955905178, 6281826),
 ('Na3', 0.5120225422667501, 143730),
 ('g3', 0.5033715727067002, 2752573),
 ('b3', 0.5030213830888164, 1471677)]

In [104]:
second_moves = {}
for move in fmlb1:
    mw1 = move[0].split(" ")[0]
    mb1 = move[0].split(" ")[1]
    if mw1 in ffmlw1:
        if mw1 in second_moves:
            second_moves[mw1].append((mb1,move[1],move[2]))
        else:
            second_moves[mw1] = [(mb1,move[1],move[2])]

In [105]:
third_moves = {}
for move in fmlw2:
    fmw = move[0].split(" ")[0]
    fmb = move[0].split(" ")[1]
    smw = move[0].split(" ")[2]
    
    if fmw+" "+fmb in ffmlb1:
        if fmw+fmb in third_moves:
            third_moves[fmw+fmb] = third_moves[fmw+fmb] + ([(smw,move[1],move[2])])
        else:
            third_moves[fmw+fmb] = [(smw,move[1],move[2])]

In [106]:
forth_moves = {}
for move in fmlb2:
    fmw = move[0].split(" ")[0]
    fmb = move[0].split(" ")[1]
    smw = move[0].split(" ")[2]
    smb = move[0].split(" ")[3]
    
    if fmw+" "+fmb+" "+smw in ffmlw2:
        if fmw+fmb+smw in forth_moves:
            forth_moves[fmw+fmb+smw] = forth_moves[fmw+fmb+smw] + ([(smb,move[1],move[2])])
        else:
            forth_moves[fmw+fmb+smw] = [(smb,move[1],move[2])]

In [107]:
fmlw1

[('Nh3', 0.5712763079410298, 613598),
 ('g4', 0.5593962228771996, 511977),
 ('c4', 0.5575896103276872, 671128),
 ('e3', 0.5400545955905178, 6281826),
 ('Na3', 0.5120225422667501, 143730),
 ('g3', 0.5033715727067002, 2752573),
 ('b3', 0.5030213830888164, 1471677)]

In [108]:
second_moves

{'Nh3': [('b6', 0.5233891279252807, 76232), ('e6', 0.4909453202836547, 85315)],
 'Na3': [('b5', 0.522741518942908, 79766), ('e6', 0.43458378859617175, 24711)],
 'g3': [('b6', 0.5142890497314966, 642450),
  ('e6', 0.5079845259703353, 272715),
  ('b5', 0.4958576700310369, 81838),
  ('g5', 0.481261861764264, 326153),
  ('g6', 0.4464190490700737, 735112),
  ('Nh6', 0.4197372307771342, 155726),
  ('c5', 0.41346334331130274, 101223)],
 'g4': [('b6', 0.508440257030104, 48399), ('g6', 0.43230419422730004, 106432)],
 'b3': [('g6', 0.5080923761298566, 295834),
  ('b5', 0.470743616835756, 194966),
  ('Na6', 0.4641160949868074, 41690),
  ('b6', 0.4623230805724678, 354116),
  ('e6', 0.45675518025688905, 255441),
  ('c5', 0.39718369198859216, 61712)],
 'e3': [('b5', 0.5002405564398114, 4287975),
  ('g5', 0.4959632008882544, 63045),
  ('c5', 0.4866542046567225, 314556)],
 'c4': [('g6', 0.4939559529723464, 54351),
  ('c5', 0.4887156775907883, 28225),
  ('e6', 0.48395434612145327, 34433),
  ('c6', 0.47

In [109]:
third_moves

{'c4g6': [('a4', 0.6381637028629084, 7929),
  ('e3', 0.5451866404715128, 6108),
  ('c5', 0.5006047899362954, 12401)],
 'b3b6': [('c4', 0.6359555079067274, 14924),
  ('Na3', 0.5247058823529411, 9775),
  ('e3', 0.5061101332662811, 19885),
  ('b4', 0.4909090909090909, 37840)],
 'g3e6': [('b4', 0.6316940149021786, 169237)],
 'b3g6': [('e3', 0.6262023789001896, 29005),
  ('c4', 0.5560526969922943, 8046),
  ('Na3', 0.5364141084258655, 6124),
  ('g4', 0.5229966634099582, 54247),
  ('g3', 0.5040465690597989, 141725)],
 'g3b6': [('c4', 0.6169710650523291, 19492),
  ('Nh3', 0.6117384179949352, 33565),
  ('b3', 0.5372625389295971, 395907),
  ('b4', 0.49710701346455416, 73972)],
 'Nh3b6': [('Nf4', 0.6018914658860617, 4441),
  ('g3', 0.5712875885289679, 34452)],
 'g3b5': [('Nh3', 0.5986410496719775, 4268),
  ('b3', 0.5488695284116962, 28351)],
 'e3c5': [('Bd3', 0.5886844526218952, 21740),
  ('Bb5', 0.513766309391662, 54953),
  ('Bc4', 0.4956475863888156, 7582)],
 'g4b6': [('g5', 0.5592344598513207,

In [110]:
forth_moves

{'b3b6Na3': [('g6', 0.5496535796766744, 2598),
  ('b5', 0.5089748703629836, 5014)],
 'b3g6Na3': [('b5', 0.5488435374149659, 3675)],
 'Na3b5Nxb5': [('d6', 0.5484496124031008, 2064),
  ('a6', 0.5210205316821082, 21479),
  ('g5', 0.5148188803512623, 911),
  ('e6', 0.5055884286653517, 4563),
  ('g6', 0.4986753017368266, 6794),
  ('c6', 0.49092588926285224, 15153),
  ('Bb7', 0.4844649021864212, 8690),
  ('Nh6', 0.4797979797979798, 1386)],
 'g3c5Nh3': [('g5', 0.547707100591716, 2704), ('c4', 0.500438596491228, 2280)],
 'g3g5b3': [('h5', 0.5456081081081081, 1184),
  ('b5', 0.5452996646880731, 92153),
  ('b6', 0.5388299851190477, 21504),
  ('Nh6', 0.5020785219399538, 2165),
  ('a6', 0.4605263157894737, 836),
  ('c5', 0.43672537508153947, 9198)],
 'c4g6c5': [('d6', 0.5450292397660819, 855),
  ('c6', 0.5260232440626579, 5937),
  ('b6', 0.4218248509857863, 4362)],
 'g3b6b3': [('Nh6', 0.5446623093681917, 2754),
  ('g6', 0.5417104433704559, 233191),
  ('e5', 0.5404651162790698, 2150),
  ('g5', 0.50

In [8]:
first_move_white = {}
first_move_black = {}
second_move_white = {}
second_move_black = {}

wins_white,wins_black,draws,not_finished,all_games = 0,0,0,0,0
w1,b1,d1 = 0,0,0
move0 = 0
move1 = 0
move2 = 0
for games in all_games_array:
    print(len(games))
    for game in games:
        if "Event" in game:
            try:
                s = game.split("Result")
                s1 = s[1].split("]")
                result = s1[0]
                white = result.split("-")[0].replace('"',"").replace(" ","")
                black = result.split("-")[1].replace('"',"").replace(" ","")

                if white == '1/2':
                    white = 0.5
                    black = 0.5
                else:
                    white = float(white)
                    black = float(black)

                if white == 1.0:
                    wins_white += 1
                    w1 = 1
                    b1 = 0
                    d1 = 0
                elif white == 0.0:
                    wins_black += 1
                    w1 = 0
                    b1 = 1
                    d1 = 0
                else:
                    draws += 1 
                    w1 = 0
                    b1 = 0
                    d1 = 1
                
            except:
                not_finished += 1            

        else:
            try:
                parsed_game = re.sub("[\(\[].*?[\)\]]", "", game).replace("{  }","").replace("!","").replace("?","").replace("...","").split(".")
                
                for move_i,move in enumerate(parsed_game[1:-1]):
                    
                    parsed_move = move.split(" ")[1:-1]
                    parsed_move = list(filter(None, parsed_move))

                    if len(parsed_move) > 3:
                        white_move = parsed_move[0]
                        black_move = parsed_move[4]
                    elif len(parsed_move) == 3:
                        white_move = parsed_move[0]
                        black_move = parsed_move[2]
                    else:
                        white_move = parsed_move[0]
                        black_move = parsed_move[1]
                    
                    if move_i == 0:
                        
                        move0 = white_move+" "+black_move
                        
                        if white_move in first_move_white:
                            score = first_move_white[white_move]
                            first_move_white[white_move] = (score[0]+w1,score[1]+b1,score[2]+d1)
                        else:
                            first_move_white[white_move] = (w1,b1,d1)
                            
                      
                        if white_move+" "+black_move in first_move_black:
                            score = first_move_black[white_move+" "+black_move]
                            first_move_black[white_move+" "+black_move] = (score[0]+w1,score[1]+b1,score[2]+d1)
                        else:
                            first_move_black[white_move+" "+black_move] = (w1,b1,d1)

                    elif move_i == 1:

                        if move0 +" "+white_move in second_move_white:
                            score = second_move_white[move0 +" "+white_move]
                            second_move_white[move0 +" "+white_move] = (score[0]+w1,score[1]+b1,score[2]+d1)
                        else:
                            second_move_white[move0 +" "+white_move] = (w1,b1,d1)

                        if move0 +" "+white_move+" "+black_move in second_move_black:
                            score = second_move_black[move0 +" "+white_move+" "+black_move]
                            second_move_black[move0 +" "+white_move+" "+black_move] = (score[0]+w1,score[1]+b1,score[2]+d1)
                        else:
                            second_move_black[move0 +" "+white_move+" "+black_move] = (w1,b1,d1)

                            
                                
                    else:
                        break

            except Exception as x:
                print("except1")
                print(x)
                
        all_games += 1


print("--------")
print(all_games)
print(not_finished)
print(wins_white)
print(wins_black)
print(draws)

59197
158713
143031
128625
119511
137349
110477
99171
110747
116129
118353
146257
167219
183085
172339
196145
217321
247313
251341
241727
232565
210455
249951
262879
272063
294951
269347
305817
290477
299863
264177
262185
248909
253441
267191
308135
297013
308407
292305
309447
305859
389711
304069
287993
284139
267179
278243
274659
345455
336661
316883
343809
332339
340367
302357
305703
296549
280587
303929
301529
312127
314837
315257
392401
626197
794923
755581
781335
732137
701645
739903
891969
904123
853479
964891
952729
1070573
891787
798881
758353
738651
708937
689805
--------
31512169
5406
7872035
7605474
273128


In [17]:
def from_canonical_to_actual(self,canonical_board):
        c_b = canonical_board
        pos = c_b[5:]
        ply = c_b[:5]

        board = []
        final_board = np.zeros((8,8)).astype(int)
        counter = 0
        i = 0
        for _ in range(len(pos)-5):
            if i >= len(pos):
                break
            piece = 0
            if pos[i] == "-":
                piece = pos[i]+pos[i+1]
                i = i + 1
            else:
                piece = pos[i]
            i = i + 1
            board.append(int(piece))

        for j in range(0,64,8):
            final_board[counter] = board[j:j+8]
            counter += 1

        square_at_index = 0
        dict_board = {}
        for i in reversed(range(0, 8)):
            for j in range(0, 8):
                piece = final_board[i][j]

                if piece != 0:

                    dict_board[square_at_index] = chess.Piece.from_symbol(REVERSE_PIECE_MAP[piece])

                square_at_index = square_at_index + 1
        temp_board = chess.variant.AntichessBoard()
        temp_board.set_piece_map(dict_board)

        return temp_board
    
    
def opening_book_first_white(book):
    board = chess.variant.AntichessBoard()
    total_moves = sum([move[2] for move in book])
    probs = np.zeros(4096)
    move_count = 0
    pre_moves = []
    for move in book:
        prob = move[2]/total_moves
        pre_moves.append((get_action_from_move(str(board.parse_san(move[0]))),prob))

    for pre_move in pre_moves:
        probs[pre_move[0]] = pre_move[1]
    return probs,board
            

In [1]:
words = open("german_propositions.txt",encoding='utf-8').readlines()

In [18]:
for word in words:
    if "#" in word:
        #print(word.strip("\n"))
        continue
    split = word.strip("\n").split("–")
    print(split[0].replace(" ","")+","+split[1].replace(",","-").replace('”',"").replace('“',""))

bis, until- up to- as far as
durch, through- by means of
für, for
ohne, without
gegen, against- toward
um, around- for
entlang, along
ab, from (time)
aus, from- out of
außer, except for
bei, at- near- at the house of
dank, thanks to
entgegen, contrary to
gegenüber, opposite
gemäß, according to
laut, according to
mit, with
nach, after- to
seit, since- for
von, from- of
zu, to
zufolge, according to
an, on (a vertical surface)
auf, on top of (horizontal surface)
hinter, behind
in, in- into
neben, next to
entlang, along
über, above
unter, under
vor, in front of
zwischen, between
anstatt(oftenabbreviatedtojuststatt), instead [of]
außerhalb, outside of
beiderseits, on both sides
diesseits, on this side
innerhalb, inside of
jenseits, on the other side
oberhalb, above
trotz, despite
unterhalb, below
wegen, because of
während, during
